In [1]:
!pip install -qU \
  datasets==2.14.6 \
  openai==1.2.2 \
  pinecone-client==3.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


## Dataset Download

We're going to test with a more real world use-case, with messy, imperfect data. We will use the [`jamescalam/ai-arxiv-chunked`](https://huggingface.co/datasets/jamescalam/ai-arxiv-chunked) dataset.

In [2]:
from datasets import load_dataset

data = load_dataset("jamescalam/ai-arxiv-chunked", split="train")
data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['doi', 'chunk-id', 'chunk', 'id', 'title', 'summary', 'source', 'authors', 'categories', 'comment', 'journal_ref', 'primary_category', 'published', 'updated', 'references'],
    num_rows: 41584
})

In [12]:
data = data.map(lambda x: {
    "id": f'{x["id"]}-{x["chunk-id"]}',
    "text": x["chunk"],
    "metadata": {
        "title": x["title"],
        "url": x["source"],
        "primary_category": x["primary_category"],
        "published": x["published"],
        "updated": x["updated"],
        "text": x["chunk"],
    }
})
# drop uneeded columns
data = data.remove_columns([
    "title", "summary", "source",
    "authors", "categories", "comment",
    "journal_ref", "primary_category",
    "published", "updated", "references",
    "doi", "chunk-id",
    "chunk"
])
data

Map:   0%|          | 0/41584 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'text', 'metadata'],
    num_rows: 41584
})

First we define our embedding function.

In [37]:
import openai

openai.api_key = "sk-..."

def embed(docs: list[str], name: str) -> list[list[float]]:
    res = openai.embeddings.create(
        input=docs, model=name
    )
    doc_embeds = [r.embedding for r in res.data]
    return doc_embeds

Initialize connection to Pinecone with free API key + sign up to serverless (for free, you get $100 in credits which will last _forever_)

In [4]:
from pinecone import Pinecone

pc = Pinecone(api_key="...")

Setup x2 Pinecone Serverless indices:

In [15]:
import time
from pinecone import ServerlessSpec

spec = ServerlessSpec(cloud='aws', region='us-west-2')

indexes = [
    {
        "name": "text-embedding-ada-002",
        "dimension": 1536
    },
    {
        "name": "text-embedding-3-small",
        "dimension": 1536  # 512
    },
    {
        "name": "text-embedding-3-large",
        "dimension": 3072  # 256
    }
]
# get existing indexes
existing_indexes = pc.list_indexes().names()

# check if index already exists (it shouldn't if this is first time)
for i, index in enumerate(indexes):
    if index["name"] not in existing_indexes:
        # if does not exist, create index
        pc.create_index(
            index["name"],
            dimension=index["dimension"],  # dimensionality of minilm
            metric='dotproduct',
            spec=spec
        )
        # wait for index to be initialized
        while not pc.describe_index(index["name"]).status['ready']:
            time.sleep(1)

    # connect to index
    indexes[i]["index"] = pc.Index(index["name"])
    time.sleep(1)
    # view index stats
    print(indexes[i]["index"].describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}
{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1300}},
 'total_vector_count': 1300}
{'dimension': 3072,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


Index everything

In [16]:
from tqdm.auto import tqdm

batch_size = 100  # how many embeddings we create and insert at once

for index in indexes:
    print(f"Indexing for {index['name']}")
    for i in tqdm(range(0, len(data), batch_size)):
        passed = False
        # find end of batch
        i_end = min(len(data), i+batch_size)
        # create batch
        batch = data[i:i_end]
        embeds = embed(batch["text"], name=index["name"])
        to_upsert = list(zip(batch["id"], embeds, batch["metadata"]))
        # upsert to Pinecone
        index["index"].upsert(vectors=to_upsert)

Indexing for text-embedding-ada-002


  0%|          | 0/416 [00:00<?, ?it/s]

Indexing for text-embedding-3-small


  0%|          | 0/416 [00:00<?, ?it/s]

Indexing for text-embedding-3-large


  0%|          | 0/416 [00:00<?, ?it/s]

Let's create a `get_docs` function so we can quickly compare results

In [18]:
def get_docs(query: str, top_k: int, model: int) -> list[str]:
    print(f"Getting docs with {indexes[model]['name']}")
    # encode query
    xq = embed([query], name=indexes[model]["name"])[0]
    # search pinecone index
    res = indexes[model]["index"].query(vector=xq, top_k=top_k, include_metadata=True)
    # get doc text
    docs = [x["metadata"]['text'] for x in res["matches"]]
    return docs

Try different embedding models by switching the `model` parameter:

* `0` is `text-embedding-ada-002`
* `1` is `text-embedding-3-small`
* `2` is `text-embedding-3-large`

In [35]:
doc = get_docs(
    query="what is the difference between llama and gpt-4?",
    top_k=5,
    model=2
)
print(">>>")
for d in doc:
    print(d)
    print(">>>")

Getting docs with text-embedding-3-large
>>>
to GPT-3 corresponds to the Stanford Alpaca model. From Figure 3(a), we observe that ( i) For the
“Helpfulness” criterion, GPT-4 is the clear winner with 54.12% of the votes. GPT-3 only wins 19.74%
of the time. ( ii) For the “Honesty” and “Harmlessness” criteria, the largest portion of votes goes
to the tie category, which is substantially higher than the winning categories but GPT-3 (Alpaca) is
slightly superior.
Second, we compare GPT-4-instruction-tuned LLaMA models against the teacher model GPT-4 in
Figure 3(b). The observations are quite consistent over the three criteria: GPT-4-instruction-tuned
LLaMA performs similarly to the original GPT-4. We conclude that learning from GPT-4 generated
5
60% 70% 80% 90% 100%12345BRanking Group 94% 624 : 66792% 614 : 67091% 623 : 68289% 597 : 66989% 605 : 67891% 609 : 666
>>>
based on the Transformer architecture [VSP+17] and trained on massive corpora of web-text data, using at its
core a self-super